---
title: Brownian Motion
format:
  live-html:
    toc: true
    toc-location: right
pyodide:
  packages:
    - numpy
    - matplotlib
---


We will apply our newly acquired knowledge about classes to simulate Brownian motion. This task aligns perfectly with the principles of object-oriented programming, as each Brownian particle (or colloid) can be represented as an object instantiated from the same class, albeit with different properties. For instance, some particles might be larger while others are smaller. We have already touched on some aspects of this in previous lectures.

```{pyodide}
#| autorun: true
#| edit: false
#| echo: false
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

plt.rcParams.update({'font.size': 12,
                     'lines.linewidth': 1,
                     'lines.markersize': 10,
                     'axes.labelsize': 11,
                     'xtick.labelsize' : 10,
                     'ytick.labelsize' : 10,
                     'xtick.top' : True,
                     'xtick.direction' : 'in',
                     'ytick.right' : True,
                     'ytick.direction' : 'in',})
```


## Brownian Motion

### What is Brownian Motion?
Imagine a dust particle floating in water. If you look at it under a microscope, you'll see it moving in a random, zigzag pattern. This is Brownian motion!


### Why Does This Happen?
When we observe Brownian motion, we're seeing the effects of countless molecular collisions. Water isn't just a smooth, continuous fluid - it's made up of countless tiny molecules that are in constant motion. These water molecules are continuously colliding with our particle from all directions. Each individual collision causes the particle to move just a tiny bit, barely noticeable on its own. However, when millions of these tiny collisions happen every second from random directions, they create the distinctive zigzag motion we observe.

::: {#fig_brownian .content-visible when-profile="default"}
![](img/Brownian Motion.gif){fig-align="center" width=40%}

Animation of Brownian motion (c) Wikipedia.
:::

### The Simplified Math Behind It
When our particle moves:

1. Each step is random in direction
2. The size of each step depends on:
   - Temperature (warmer = more movement)
   - Time between steps
   - A property called the "diffusion coefficient" (D)

### How We Can Simulate This?
In Python, we can simulate these random steps using random number. These random numbers can be generated with the numpy library. Numpy provides a number of different functions that provide random numbers from different distributions. For Brownian motion, we use a special distribution called the "normal distribution".

```python
step_size = np.sqrt(2 * D * time_step)
dx = random_number * step_size  # Random step in x direction
dy = random_number * step_size  # Random step in y direction

new_x = old_x + dx
new_y = old_y + dy
```

Where:

- `D` is how easily the particle moves (diffusion coefficient)
- `time_step` is how often we update the position
- `random_number` is chosen from a special "normal distribution"

::: {.callout-tip}
When simulating Brownian motion, we use `np.random.normal` to generate random steps following this distribution. The normal distribution is characterized by two parameters: the mean and the standard deviation. The mean is the average value, and the standard deviation is a measure of how spread out the values are. For Brownian motion, we use a standard deviation that depends on the diffusion coefficient and the time step. The standard deviation $\sigma=\sqrt{2D \Delta t}$ determines the typical step size, which we can use as a parameter in the normal distribution.
:::


```{pyodide}
#| autorun: false

# some space to test out some of the random numbers




```


::: {.callout-note collapse="true"}
## Advanced Mathematical Details
The Brownian motion of a colloidal particle results from collisions with surrounding solvent molecules. These collisions lead to a probability distribution described by:

$$
p(x,\Delta t)=\frac{1}{\sqrt{4\pi D \Delta t}}e^{-\frac{x^2}{4D \Delta t}}
$$

where:
- $D$ is the diffusion coefficient
- $\Delta t$ is the time step
- The variance is $\sigma^2=2D \Delta t$

This distribution emerges from the **central limit theorem**, as shown by Lindenberg and Lévy, when considering many infinitesimally small random steps.

The evolution of the probability density function $p(x,t)$ is governed by the diffusion equation:

$$
\frac{\partial p}{\partial t}=D\frac{\partial^2 p}{\partial x^2}
$$

This partial differential equation, also known as Fick's second law, describes how the concentration of particles evolves over time due to diffusive processes. The Gaussian distribution above is the fundamental solution (Green's function) of this diffusion equation, representing how an initially localized distribution spreads out over time.

The connection between the microscopic random motion and the macroscopic diffusion equation was first established by Einstein in his 1905 paper on Brownian motion, providing one of the earliest quantitative links between statistical mechanics and thermodynamics.
:::


## Why Use a Class?

A class is perfect for this physics simulation because each colloidal particle:

1. Has specific properties
   - Size (radius)
   - Current position
   - Movement history
   - Diffusion coefficient

2. Follows certain behaviors
   - Moves randomly (Brownian motion)
   - Updates its position over time
   - Keeps track of where it's been

3. Can exist alongside other particles
   - Many particles can move independently
   - Each particle keeps track of its own properties
   - Particles can have different sizes

4. Needs to track its state over time
   - Remember previous positions
   - Calculate distances moved
   - Maintain its own trajectory

This natural mapping between real particles and code objects makes classes an ideal choice for our simulation.

## Class Design

Let's design a Python class to simulate colloidal particles undergoing Brownian motion. This object-oriented approach will help us manage multiple particles with different properties and behaviors.

### Class-Level Properties
The `Colloid` class will maintain information shared by all particles:

1. A counter for the total number of particles
2. The physical constant $k_B T/(6\pi\eta) = 2.2×10^{-19}$ (combining temperature and fluid properties)

### Class Methods
The class will provide these shared functions:

1. `how_many()`: Reports the total number of particles
2. `__str__`: Creates a readable description of a particle's properties

### Instance Properties
Each individual particle object will have:

1. Radius (R)
2. Position history (x and y coordinates)
3. Unique identifier (index)
4. Diffusion coefficient ($D = k_B T/(6\pi\eta R)$)

### Instance Methods
Each particle will be able to:

1. `sim_trajectory()`: Generate a complete motion path
2. `update(dt)`: Calculate one step of Brownian motion
3. `get_trajectory()`: Return its movement history
4. `get_D()`: Provide its diffusion coefficient


```{pyodide}
#| autorun: false
# Class definition
class Colloid:

    # A class variable, counting the number of Colloids
    number = 0
    f = 2.2e-19 # this is k_B T/(6 pi eta) in m^3/s

    # constructor
    def __init__(self,R, x0=0, y0=0):
        # add initialisation code here
        self.R=R
        self.x=[x0]
        self.y=[y0]
        Colloid.number=Colloid.number+1
        self.index=Colloid.number
        self.D=Colloid.f/self.R

    def get_D(self):
        return(self.D)

    def sim_trajectory(self,N,dt):
        for i in range(N):
            self.update(dt)

    def update(self,dt):
        self.x.append(self.x[-1]+np.random.normal(0.0, np.sqrt(2*self.D*dt)))
        self.y.append(self.y[-1]+np.random.normal(0.0, np.sqrt(2*self.D*dt)))
        return(self.x[-1],self.y[-1])

    def get_trajectory(self):
        return(pd.DataFrame({'x':self.x,'y':self.y}))

    # class method accessing a class variable
    @classmethod
    def how_many(cls):
        return(Colloid.number)

    # insert something that prints the particle position in a formatted way when printing
    def __str__(self):
        return("I'm a particle with radius R={0:0.3e} at x={1:0.3e},y={2:0.3e}.".format(self.R, self.x[-1], self.y[-1]))
```

::: {.callout-note}
## Note

Note that the function `sim_trajectory` is actually calling the function `update` of the same object to generate the whole trajectory at once.
:::

## Simulating

With the help of this Colloid class, we would like to carry out simulations of Brownian motion of multiple particles. The simulations shall

* take n=200 particles
* have N=200 trajectory points each
* start all at 0,0
* particle objects should be stored in a list p_list

```{pyodide}
#| autorun: false
N=200 # the number of trajectory points
n=200 # the number of particles

p_list=[]
dt=0.05

# creating all objects
for i in range(n):
    p_list.append(Colloid(1e-6))


for (index,p) in enumerate(p_list):
    p.sim_trajectory(N,dt)
```

```{pyodide}
#| autorun: false
print(p_list[42])
```

## Plotting the trajectories

The next step is to plot all the trajectories.

```{pyodide}
#| autorun: false
# we take real world diffusion coefficients so scale up the data to avoid nasty exponentials
scale=1e6

plt.figure(figsize=(4,4))

[plt.plot(np.array(p.x[:])*scale,np.array(p.y[:])*scale,'k-',alpha=0.1,lw=1) for p in p_list]
plt.xlim(-10,10)
plt.ylim(-10,10)
plt.xlabel('x [µm]')
plt.ylabel('y [µm]')
plt.tight_layout()
plt.show()
```

## Characterizing the Brownian motion

Now that we have a number of trajectories, we can analyze the motion of our Brownian particles.

### Calculate the particle speed

One way is to calculate its speed by measuring how far it traveled within a certain time $n\, dt$, where $dt$ is the timestep of out simulation. We can do that as

\begin{equation}
v(n dt) = \frac{<\sqrt{(x_{i+n}-x_{i})^2+(y_{i+n}-y_{i})^2}>}{n\,dt}
\end{equation}

The angular brackets on the top take care of the fact that we can measure the distance traveled within a certain time $n\, dt$ several times along a trajectory.

These values can be used to calculate a mean speed. Note that there is not an equal amount of data pairs for all separations available. For $n=1$ there are 5 distances available. For $n=5$, however, only 1. This changes the statistical accuracy of the mean.

```{pyodide}
#| autorun: false
time=np.array(range(1,N))*dt

plt.figure(figsize=(4,4))
for j in range(100):
    t=p_list[j].get_trajectory()
    md=[np.mean(np.sqrt(t.x.diff(i)**2+t.y.diff(i)**2)) for i in range(1,N)]
    md=md/time
    plt.plot(time,md,alpha=0.4)

plt.ylabel('speed [m/s]')
plt.xlabel('time [s]')
plt.tight_layout()
plt.show()
```

The result of this analysis shows, that each particle has an apparent speed which seems to increase with decreasing time of observation or which decreases with increasing time. This would mean that there is some friction at work, which slows down the particle in time, but this is apparently not true. Also an infinite speed at zero time appears to be unphysical.
The correct answer is just that the speed is no good measure to characterize the motion of a Brownian particle.

### Calculate the particle mean squared displacement

A better way to characterize the motion of a Brownian particle is the mean squared displacement, as we have already mentioned it in previous lectures. We may compare our simulation now to the theoretical prediction, which is

\begin{equation}
\langle \Delta r^{2}(t)\rangle=2 d D t
\end{equation}

where $d$ is the dimension of the random walk, which is $d=2$ in our case.

```{pyodide}
#| autorun: false
time=np.array(range(1,N))*dt

plt.figure(figsize=(4,4))
for j in range(100):
    t=p_list[j].get_trajectory()
    msd=[np.mean(t.x.diff(i).dropna()**2+t.y.diff(i).dropna()**2) for i in range(1,N)]
    plt.plot(time,msd,alpha=0.4)


plt.plot(time, 4*p_list[0].D*time,'k--',lw=2,label='theory')
plt.legend()
plt.xlabel('time [s]')
plt.ylabel('msd $[m^2/s]$')
plt.tight_layout()
plt.show()
```

The results show that the mean squared displacement of the individual particles follows *on average* the theoretical predictions of a linear growth in time. That means, we are able to read the diffusion coefficient from the slope of the MSD of the individual particles if recorded in a simulation or an experiment.

Yet, each individual MSD is deviating strongly from the theoretical prediction especially at large times. This is due to the fact mentioned earlier that our simulation (or experimental) data only has a limited number of data points, while the theoretical prediction is made for the limit of infinite data points.


::: {.callout-warning}
## Analysis of MSD data

Single particle tracking, either in the experiment or in numerical simulations can therefore only deliver an estimate of the diffusion coefficient and care should be taken when using the whole MSD to obtain the diffusion coefficient. One typically uses only a short fraction of the whole MSD data at short times.
:::